### Enter full names of group members:

##### Name A:
##### Name B:

In [435]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [436]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [437]:
from collections import deque
import math

def dgim_algorithm(stream_path, N):
    # Create the buckets and initialize the timestamp
    buckets = deque(maxlen=int(math.log2(N)) + 1)
    timestamp = 0
    
    # Helper function to merge buckets
    def merge_buckets():
        nonlocal buckets
        while len(buckets) > 2:
            if len(buckets[0]) == len(buckets[1]):
                merged_bucket = deque(buckets[0] + buckets[1], maxlen=None)
                buckets.popleft()
                buckets.popleft()
                while len(buckets) >= 2 and len(merged_bucket) == len(buckets[0]) == len(buckets[1]):
                    merged_bucket.extend(buckets[0] + buckets[1])
                    buckets.popleft()
                    buckets.popleft()
                buckets.appendleft(merged_bucket)
            else:
                break
    
    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        count = 0
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break
            
            # Update timestamp
            timestamp = (timestamp + 1) % N
            
            # Add new bucket with current timestamp if bit is 1
            if bit == '1':
                count += 1
                buckets.append(deque([timestamp], maxlen=None))
                merge_buckets()
                count %= N  # Ensure count doesn't grow beyond N
    
    # Adjust buckets to conform to rules and format output
    output_buckets = []
    last_bucket = []
    for bucket in buckets:
        if bucket:
            # Filter out timestamps according to the specified rules
            filtered_bucket = [ts for ts in bucket if ts > timestamp - N]
            if filtered_bucket:
                output_buckets.append(tuple(filtered_bucket))
                last_bucket = list(filtered_bucket)
    
    # Add empty buckets if needed to ensure the total number of buckets is maintained
    while len(output_buckets) < int(math.log2(N)) + 1:
        output_buckets.append(())
    
    # Find the end timestamp
    end_time_stamp = max(last_bucket) if last_bucket else 0
    
    return output_buckets, end_time_stamp


In [438]:
bucket = dgim_algorithm(stream_path, N)


In [439]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [(84, 85), (88,), (89,), (90,), (91,), (94,), (96,), (99,), ()]
The end timestamp: 99


#### 1.2. Query the Bucket 

In [440]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [441]:
def dgim_query(bucket, N, k):  
    
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
   
    
    # To-do! initialize the different variables
    

    # To-do! query the dgim bucket using the k parameters
    
    
    return math.ceil(one_count)

In [442]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [443]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------


NameError: name 'one_count' is not defined

### 2. Bloom filters

In [ ]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [ ]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [ ]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [ ]:
def generate_hash(h, N):
    hash_list = []
    primes = [114649, 914843, 1382753]
    for j in range(h):
        hash_list.append(lambda s, j=j: sum(ord(s[i]) * primes[j]**i for i in range(len(s))) % N)
    return hash_list

In [ ]:
hashes = generate_hash(h, bloom_size)

In [ ]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            for hash in hashes:
                B[hash(name)] = 1
    return B

In [ ]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [ ]:
bloom_array

array([0., 1., 0., ..., 1., 0., 0.])

#### 2.2. Verify usernames

In [ ]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    for hash in hashes:
        if (bloom_array[hash(new_user)] == 0):
            return 0
    return 1
    

In [ ]:
# Feel free to test different usernames here

# new_username = "KazeemTDT4305"

new_username = "sanna86h88hy"

In [ ]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [ ]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username sanna86h88hy is available. Congrats!


In [ ]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            hash_hit_count = 0
            for hash in hashes:
                if (bloom_array[hash(name)] == 0):
                    break
                elif (bloom_array[hash(name)] == 1):
                    hash_hit_count += 1
            total_name += 1
            if (hash_hit_count == 3):
                taken_name += 1
            
    return round(taken_name/total_name*100,2)   

In [ ]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 24.0%
----------------------------------------------------------


### 3. Flajolet-Martin

In [ ]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    hash_func = lambda x: (6 * x + 1) % 5

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    for element in input_stream:
        hash_value = hash_func(element)
        print(hash_value)
        binary_hash = bin(hash_value)[2:]
        print(binary_hash)

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [ ]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

2
10
2
10
3
11
2
10
3
11
2
10
2
10
2
10
2
10
3
11
2
10
2
10
2
10
4
100
3
11
2
10
3
11
4
100
0
0
4
100
2
10
3
11
4
100
2
10
-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 1
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 1
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [684]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [685]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [686]:
import copy
import random

def greedy_algorithm(local_companies, queries):
    # setter revenue til 0
    revenue = 0
    
    # For-løkke gjennom spørringene som setter selskapene med ord som matcher i en liste
    for q in queries:
        matched_companies = []
        
        # Company er A, B, C, D og data er tilhørende data i local_companies dictionary.
        for company, data in local_companies.items():
            keywords = data[:-1]  # Keywords er alle elementene utenom det siste derfor det er :-1
            budget = data[-1]      # budget er det siste elementet derfor -1
            # legger til elementene som har $$
            if budget > 0 and q in keywords:
                matched_companies.append(company)
    
        # Velger bare et random selskap fra listen som matcher og har budsjett over 0
        if matched_companies:
            selected_company = random.choice(matched_companies)
            local_companies[selected_company][-1] -= 1  #Trekker fra 1 i budsjettet
            revenue += 1
    
    return revenue





In [672]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 8
Trial 2 - Revenue generated: 10
Trial 3 - Revenue generated: 8
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 7
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 11
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 10
Trial 10 - Revenue generated: 10
------------------------------------------------
Average revenue generated for all trials:  9.1


#### 4.2. Balance Algorithm

In [675]:

#blabla
def balance_algorithm(local_companies, queries):
    #Helt lik start på koden
    revenue = 0
    
    for q in queries:
        matched_companies = []

        for company, data in local_companies.items():
            keywords = data[:-1]  
            budget = data[-1]     
            
            if budget > 0 and q in keywords:
                matched_companies.append((company, budget))  # Legger også til budsjett i listen
        
        if matched_companies:
            # Sorterer matchene etter hvor mye $ de har og finner den med mest
            matched_companies.sort(key=lambda x: x[1], reverse=True)
            max_budget = matched_companies[0][1]
            #Tar ut de som har det høyeste budsejttet
            max_budget_companies = [company for company, budget in matched_companies if budget == max_budget]
            # Hvis det er mer enn 1 velger man random en av de
            selected_company = random.choice(max_budget_companies)
            
            # trekker fra 1 i budsjettet
            if local_companies[selected_company][-1] > 0:
                local_companies[selected_company][-1] -= 1
                revenue += 1  
    
    return revenue




In [683]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 8
Trial 2 - Revenue generated: 8
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 10
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 9
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 10
Trial 10 - Revenue generated: 10
-------------------------------------------
Average revenue generated for all trials:  9.3


### 5. Recommender System

In [ ]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [ ]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    
    return prediction   

In [ ]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [ ]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    
    return prediction

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [ ]:
# Enter answer here

#### Case 2

In [ ]:
# Enter answer here

#### Case 3

In [ ]:
# Enter answer here

#### Case 4

In [ ]:
# Enter answer here

#### Case 5

In [ ]:
# Enter answer here